In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns

# Getting to know the data

In [2]:
df_train = pd.read_csv('precos_imoveis.csv')
df_train['is_test'] = pd.Series(np.zeros(1460))
df_train.shape

(1460, 82)

In [3]:
y = np.log(df_train['SalePrice'])
del df_train['SalePrice']

In [4]:
df_test = pd.read_csv('test.csv')
df_test['is_test'] = pd.Series(np.zeros(1460)+1)
df_test.shape

(1459, 81)

In [5]:
df = pd.concat([df_train,df_test],sort=False)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,is_test
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,0.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,0.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,0.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,0.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,0.0


In [6]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [7]:
is_test = df.is_test

# Some feature engineering

In [8]:
#it is important to know the time from the last remod
df['LastRemod'] = df['YearRemodAdd'] - df['YearBuilt']

#total number of living square feets
df['UsableSF'] = df['1stFlrSF'] + df['2ndFlrSF']

In [11]:
#For sure 'Id' shouldn't be a feature and 'SalePrice' is what we want to predict
del df['Id'],df['is_test']

# I have to process the features

In [13]:
#piece of code that tries to get the discrete features
def get_discrete_features(df):
    
    discrete_features = []
    is_discrete = []
    d = df.dtypes.to_dict()
    for column in df.columns:
        #if the type of the column is a string
        if d[column] == 'O':
            discrete_features.append(column)
            is_discrete.append(True)
        #if all unique values are the only possible values
        elif (len(df[column].unique()) == df[column].max()) or (len(df[column].unique()) == df[column].max()-1):
            discrete_features.append(column)
            is_discrete.append(True)
        else:
            is_discrete.append(False)
            
    is_discrete = np.array(is_discrete)
    return discrete_features,is_discrete

In [14]:
discrete_features,is_discrete = get_discrete_features(df)
#I know MSSubClass is discrete and it fails my test
discrete_features.append('MSSubClass')
is_discrete[0] = True

In [15]:
#Looking at the continuous variables
df[df.columns[~is_discrete]].head()

,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,YrSold,LastRemod,UsableSF
0,65.0,8450,2003,2003,196.0,706.0,0.0,150.0,856.0,856,...,0,61,0,0,0,0,0,2008,0,1710
1,80.0,9600,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,...,298,0,0,0,0,0,0,2007,0,1262
2,68.0,11250,2001,2002,162.0,486.0,0.0,434.0,920.0,920,...,0,42,0,0,0,0,0,2008,1,1786
3,60.0,9550,1915,1970,0.0,216.0,0.0,540.0,756.0,961,...,0,35,272,0,0,0,0,2006,55,1717
4,84.0,14260,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,...,192,84,0,0,0,0,0,2008,0,2198


## First, dealing with the continuous variables

In [16]:
#I want to add a new column that says a given feature was NaN and substitute the NaN for the mean
for column in df.columns[~is_discrete]:
    has_na = df[column].isna().sum() > 0
    if has_na:
        x = df[column].values[~df[column].isna()]
        #replaces NaNs with the median
        df[column] = df[column].fillna(np.median(x))
        #adds a new column saying the value was missing
        df[column + '_missing'] = df[column].isna()

In [17]:
df.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [21]:
#I added three discrete columns
is_discrete=np.append(is_discrete,[True]*11)
discrete_features.extend(list(df.columns.values[-11:]))

## Now dealing with the discrete variables. First try to encode with dummy encoding

In [49]:
df_d = df[discrete_features]
df_d = df_d.fillna('NA')
df_d.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,MasVnrArea_missing,BsmtFinSF1_missing,BsmtFinSF2_missing,BsmtUnfSF_missing,TotalBsmtSF_missing,BsmtFullBath_missing,BsmtHalfBath_missing,GarageYrBlt_missing,GarageCars_missing,GarageArea_missing
0,RL,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,False,False,False,False,False,False,False,False,False,False
1,RL,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,False,False,False,False,False,False,False,False,False,False
2,RL,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,False,False,False,False,False,False,False,False,False,False
3,RL,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,False,False,False,False,False,False,False,False,False,False
4,RL,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,False,False,False,False,False,False,False,False,False,False


In [51]:
#Making the custom encoding in the apropriate features
df_d['Street']=df_d['Street'].map({'Pave':0,'Grvl':1})
df_d['Alley']=df_d['Alley'].map({'NA':0,'Pave':1,'Grvl':2})
df_d['LotShape']=df_d['LotShape'].map({'IR3':0,'IR2':1,'IR1':2,'Reg':3})
df_d['LandContour']=df_d['LandContour'].map({'Low':0,'HLS':1,'Bnk':2,'Lvl':3})
df_d['Utilities']=df_d['Utilities'].map({'NA':-1,'ELO':0,'NoSeWa':1,'NoSewr':2,'AllPub':3})
df_d['LandSlope']=df_d['LandSlope'].map({'Sev':0,'Mod':1,'Gtl':2})
df_d['ExterQual']=df_d['ExterQual'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['ExterCond']=df_d['ExterCond'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['BsmtQual']=df_d['BsmtQual'].map({'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['BsmtCond']=df_d['BsmtCond'].map({'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['BsmtExposure']=df_d['BsmtExposure'].map({'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4})
df_d['BsmtFinType1']=df_d['BsmtFinType1'].map({'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6})
df_d['BsmtFinType2']=df_d['BsmtFinType2'].map({'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6})
df_d['HeatingQC']=df_d['HeatingQC'].map({'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['Electrical']=df_d['Electrical'].map({'NA':-1,'Mix':0,'FuseP':1,'FuseF':2,'FuseA':3,'SBrkr':4})
df_d['KitchenQual']=df_d['KitchenQual'].map({'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['FireplaceQu']=df_d['FireplaceQu'].map({'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['GarageFinish']=df_d['GarageFinish'].map({'NA':0,'Unf':1,'RFn':2,'Fin':3})
df_d['GarageCond']=df_d['GarageCond'].map({'NA':-1,'Po':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['PavedDrive']=df_d['PavedDrive'].map({'N':0,'P':1,'Y':2})
df_d['PoolQC']=df_d['PoolQC'].map({'NA':0,'Fa':1,'TA':2,'Gd':3,'Ex':4})
df_d['Fence']=df_d['Fence'].map({'NA':0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4})


In [52]:
df_d.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,MasVnrArea_missing,BsmtFinSF1_missing,BsmtFinSF2_missing,BsmtUnfSF_missing,TotalBsmtSF_missing,BsmtFullBath_missing,BsmtHalfBath_missing,GarageYrBlt_missing,GarageCars_missing,GarageArea_missing
0,RL,0,0,3,3,3,Inside,2,CollgCr,Norm,...,False,False,False,False,False,False,False,False,False,False
1,RL,0,0,3,3,3,FR2,2,Veenker,Feedr,...,False,False,False,False,False,False,False,False,False,False
2,RL,0,0,2,3,3,Inside,2,CollgCr,Norm,...,False,False,False,False,False,False,False,False,False,False
3,RL,0,0,2,3,3,Corner,2,Crawfor,Norm,...,False,False,False,False,False,False,False,False,False,False
4,RL,0,0,2,3,3,FR2,2,NoRidge,Norm,...,False,False,False,False,False,False,False,False,False,False


In [53]:
df_d = pd.get_dummies(df_d,drop_first=True)

In [54]:
X = pd.concat([df_d,df[df.columns[~is_discrete]]], axis=1)

In [57]:
X.shape

(2919, 224)

# Now lets try to fit a regressor

In [61]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
scores = cross_val_score(rf, X.loc[is_test==0], y, cv=10,scoring = 'neg_mean_squared_error')
-scores,-np.mean(scores),np.std(scores)

(array([0.02099122, 0.01467831, 0.01721415, 0.02950736, 0.02780386,
        0.0140117 , 0.0186799 , 0.01370764, 0.02172242, 0.02359078]),
 0.02019073228938422,
 0.005301385010607402)

In [62]:
ab = AdaBoostRegressor(n_estimators=100, base_estimator= DecisionTreeRegressor())
scores = cross_val_score(ab, X.loc[is_test==0], y, cv=10,scoring = 'neg_mean_squared_error')
-scores,-np.mean(scores),np.std(scores)

(array([0.02001008, 0.01410812, 0.018199  , 0.03144999, 0.02693029,
        0.01402184, 0.01930485, 0.0135608 , 0.01531582, 0.02352135]),
 0.019642214438783258,
 0.005725692610456919)

In [66]:
gb = GradientBoostingRegressor(n_estimators=100)
scores = cross_val_score(gb, X.loc[is_test==0], y, cv=10,scoring = 'neg_mean_squared_error')
-scores,-np.mean(scores),np.std(scores)

(array([0.01622209, 0.01115842, 0.01303436, 0.02657773, 0.02285309,
        0.01223938, 0.0201733 , 0.01066902, 0.0125629 , 0.01978255]),
 0.016527284450329506,
 0.005238868346721956)

# Now getting the predictions to submit

In [68]:
gb = GradientBoostingRegressor(n_estimators=100)
gb.fit(X.loc[is_test==0],y)
y_pred = gb.predict(X.loc[is_test==1])

In [69]:
ids = list(df_test.Id)

In [95]:
df_out = pd.DataFrame(list(zip(ids,np.exp(1)**y_pred)),columns=['Id','SalePrice'])

In [98]:
df_out['SalePrice'] = df_out['SalePrice'].apply(lambda x: int(x))

In [99]:
df_out.to_csv('submission.csv',index=False)

In [100]:
df_out.head()

,Id,SalePrice
0,1461,120733
1,1462,163218
2,1463,182711
3,1464,181520
4,1465,204905
